In [2]:
!pip install folium
!pip install branca

In [222]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
# import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

import branca as br 

In [223]:
#Gettting the character format

base = open(r"../data/CCPP_IGN100K.shp", 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

In [224]:
# Helpers mínimos 
norm = lambda s: ''.join(c for c in unicodedata.normalize('NFD', str(s).upper().strip())
                         if unicodedata.category(c)!='Mn')
def col_dep(gdf):
    for c in gdf.columns:
        u=c.upper()
        if any(k in u for k in ("DEPART","DPTO","DEPARTAMENTO","DEP")): return c
    return gdf.columns[0]

In [225]:
# 1) Cargamos CCPP y asegurar CRS EPSG:4326
ccpp = gpd.read_file(r"../data/CCPP_IGN100K.shp")
ccpp = ccpp.set_crs(4326) if ccpp.crs is None else ccpp.to_crs(4326)

In [46]:
ccpp.head(5)

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
0,1,PANDISHARI,INEI,2502010002,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06462,-10.37129,PANDISHARI,POINT (-74.06462 -10.37129)
1,2,CHICOSA,INEI,2502010003,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06153,-10.37852,CHICOSA,POINT (-74.06153 -10.37852)
2,3,RAYA,IGN,2502010004,OTROS,RAYMONDI,ATALAYA,UCAYALI,2350,Centro Poblado Menor,-72.94118,-10.33043,RAYA,POINT (-72.94118 -10.33043)
3,4,PENSILVANIA,INEI,2502010005,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.05988,-10.40401,PENSILVANIA,POINT (-74.05988 -10.40401)
4,5,PONTE VEDRA,INEI,2502010006,CASERÍO,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.03788,-10.41809,PONTE VEDRA,POINT (-74.03787 -10.41809)


In [226]:
#Limpieza de data 

# Usar/crear columnas X (lon) e Y (lat)
if not {"X","Y"}.issubset(ccpp.columns):
    ccpp["X"] = ccpp.geometry.x
    ccpp["Y"] = ccpp.geometry.y

# 1) Coordenadas válidas
ccpp["X"] = pd.to_numeric(ccpp["X"], errors="coerce")
ccpp["Y"] = pd.to_numeric(ccpp["Y"], errors="coerce")

valid = (
    np.isfinite(ccpp["X"]) & np.isfinite(ccpp["Y"]) &   # no NaN/inf
    ccpp["X"].between(-180, 180) &                     # rango lon
    ccpp["Y"].between(-90, 90) &                       # rango lat
    (ccpp["X"] != 0) & (ccpp["Y"] != 0)                # evita (0,0)
)

ccpp = ccpp[valid].copy()

# 2) Eliminar duplicados
ccpp = ccpp.drop_duplicates(subset=["X","Y"]).copy()
# (opcional) también por geometría:
# ccpp = ccpp.drop_duplicates(subset=["geometry"]).copy()

ccpp = ccpp.reset_index(drop=True)
print(f"Registros limpios: {len(ccpp):,}")


Registros limpios: 130,704


In [227]:
## si ccpp ya está en EPSG:4326 y es de puntos:
ccpp_cent = ccpp.copy()

# limpieza (geometrías válidas y sin duplicados exactos)
ccpp_cent = ccpp_cent[ccpp_cent.geometry.notna()].copy()
ccpp_cent = ccpp_cent.drop_duplicates(subset=["geometry"]).copy()

## A partir de los centros de población , calcule el centroide de cada localidad (o utilice los centroides geométricos proporcionados).¶

In [228]:
## Filtramos Lima 
ccpp_lima = ccpp[ccpp.DEP == "LIMA"]
ccpp_lima 

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
49200,49201,RIEGOPAMPA,IGN,None,None,SAN JOSE DE QUERO,CONCEPCION,LIMA,1746,Centro Poblado Menor,-75.54816,-12.05101,RIEGOPAMPA,POINT (-75.54816 -12.05101)
110578,113955,PUMARINRI,INEI,1503010015,UNID. AGROPECUARIA,CAJATAMBO,CAJATAMBO,LIMA,1550,Centro Poblado Menor,-76.87627,-76.87627,PUMARINRI,POINT (-76.87627 -10.4448)
110579,113956,TONGLA GRANDE,INEI,1503010016,UNID. AGROPECUARIA,CAJATAMBO,CAJATAMBO,LIMA,1550,Centro Poblado Menor,-76.89722,-76.89722,TONGLA GRANDE,POINT (-76.89722 -10.45462)
110580,113958,TOCLAPATA,IGN,1503010018,UNID. AGROPECUARIA,CAJATAMBO,CAJATAMBO,LIMA,1550,Centro Poblado Menor,-76.89709,-76.89709,TOCLAPATA,POINT (-76.89709 -10.4577)
110581,113959,ISCOPAMPA,IGN,1503010019,UNID. AGROPECUARIA,CAJATAMBO,CAJATAMBO,LIMA,1550,Centro Poblado Menor,-76.92644,-76.92644,ISCOPAMPA,POINT (-76.92644 -10.46019)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117308,121563,TUMAN,INEI,None,None,AMBAR,HUAURA,LIMA,None,None,-77.15068,-77.15068,None,POINT (-77.15068 -10.69916)
117309,121564,CASHAHUTO,INEI,None,None,AMBAR,HUAURA,LIMA,None,None,-77.14377,-77.14377,None,POINT (-77.14377 -10.69142)
117310,121565,HUALLAN,INEI,None,None,AMBAR,HUAURA,LIMA,None,None,-77.27896,-77.27896,None,POINT (-77.27896 -10.68386)
117311,121566,ARENAL PAMPA,INEI,None,None,AMBAR,HUAURA,LIMA,None,None,-77.09254,-77.09254,None,POINT (-77.09254 -10.68314)


In [229]:
# Centroide de cada localidad para Lima  
pt = ccpp_lima.unary_union.centroid
lat_lima, lon_lima = float(pt.y), float(pt.x)

m_lima = fm.Map(location=[lat_lima, lon_lima], zoom_start=9, tiles="OpenStreetMap",control_scale=True)
fm.Marker([lat_lima, lon_lima], tooltip="Centroide (Lima)").add_to(m_lima)
m_lima


/var/folders/cx/g5gs5tnx3m3bprnnzxq291_40000gn/T/ipykernel_11368/3474888477.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  pt = ccpp_lima.unary_union.centroid


In [230]:
## Filtramos para Loreto   
ccpp_loreto = ccpp[ccpp.DEP == "LORETO"]
ccpp_loreto 

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
117313,121568,PUEBLO NUEVO,INEI,1606010070,CASERÍO,CONTAMANA,UCAYALI,LORETO,1754,Centro Poblado Menor,-75.61702,-75.61702,PUEBLO NUEVO,POINT (-75.61702 -8.07216)
117314,121569,NUEVO BELLAVISTA,INEI,1606030021,CASERÍO,PADRE MARQUEZ,UCAYALI,LORETO,1954,Centro Poblado Menor,-74.87171,-74.87171,NUEVO BELLAVISTA,POINT (-74.87171 -8.00469)
117315,121570,MARISCAL CASTILLA,IGN,1606030022,CASERÍO,PADRE MARQUEZ,UCAYALI,LORETO,1954,Centro Poblado Menor,-74.78466,-74.78466,MARISCAL CASTILLA,POINT (-74.78466 -8.00247)
117316,121571,NUEVO EGIPTO,IGN,None,None,CONTAMANA,UCAYALI,LORETO,1754,Centro Poblado Menor,-75.60278,-75.60278,NUEVO EGIPTO,POINT (-75.60278 -8.01924)
117317,121572,NUEVO EDEN,IGN,None,None,CONTAMANA,UCAYALI,LORETO,1754,Centro Poblado Menor,-75.65728,-75.65728,NUEVO EDEN,POINT (-75.65728 -8.18889)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120463,125010,ESTIRON DE SANTA MARIA,INEI,None,None,TORRES CAUSANA,MAYNAS,LORETO,None,None,-74.88989,-74.88989,None,POINT (-74.88989 -1.08194)
120464,125011,SAN MARTIN DE PORRES,INEI,None,None,PUTUMAYO,MAYNAS,LORETO,None,None,-74.30302,-74.30302,None,POINT (-74.30302 -1.00774)
120465,125012,NUEVA IPIRANGA,INEI,None,None,PUTUMAYO,MAYNAS,LORETO,None,None,-74.29694,-74.29694,None,POINT (-74.29694 -0.81623)
120466,125013,PUERTO ESTRELLA,INEI,None,None,TORRES CAUSANA,MAYNAS,LORETO,None,None,-75.23722,-75.23722,None,POINT (-75.23722 -0.53257)


In [231]:
# Centroide de cada localidad para Loreto  
pt = ccpp_loreto.unary_union.centroid
lat_loreto, lon_loreto = float(pt.y), float(pt.x)

m_loreto = fm.Map(location=[lat_loreto, lon_loreto], zoom_start=9, tiles="OpenStreetMap",control_scale=True)
fm.Marker([lat_loreto, lon_loreto], tooltip="Centroide (Lima)").add_to(m_loreto)
m_loreto

/var/folders/cx/g5gs5tnx3m3bprnnzxq291_40000gn/T/ipykernel_11368/2902105935.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  pt = ccpp_loreto.unary_union.centroid


## Para cada centroide, calcule el número de hospitales operativos dentro de un área de influencia de 10 km .Identificar (por región)

In [232]:
# === 1) Cargar hospitales EN FUNCIONAMIENTO -> GeoDataFrame (EPSG:4326) ===
ip = pd.read_csv("../data/IPRESS.csv", encoding="latin-1")
ip = ip[ip["Condición"].str.upper().str.strip()=="EN FUNCIONAMIENTO"].copy()

U = lambda s: str(s).upper().strip()
loncol = next((c for c in ip.columns if U(c) in {"LONGITUD","LON","NORTE"}), None)
latcol = next((c for c in ip.columns if U(c) in {"LATITUD","LAT","ESTE"}),  None)
assert loncol and latcol, "No se hallaron columnas LONG/LAT o NORTE/ESTE en IPRESS.csv"

ip[loncol] = pd.to_numeric(ip[loncol], errors="coerce")
ip[latcol] = pd.to_numeric(ip[latcol], errors="coerce")
ip = ip.dropna(subset=[loncol, latcol]).drop_duplicates(subset=[loncol, latcol])

gdf_hosp = gpd.GeoDataFrame(ip, geometry=gpd.points_from_xy(ip[loncol], ip[latcol]), crs=4326)

In [85]:
ip.head(5)

,Institución,Código Único,Nombre del establecimiento,Clasificación,Tipo,Departamento,Provincia,Distrito,UBIGEO,Dirección,...,Inicio de Actividad,Director Médico y/o Responsable de la Atención de Salud,Estado,Situación,Condición,Inspección,NORTE,ESTE,COTA,CAMAS
1,GOBIERNO REGIONAL,7050,AMBATO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,CAJAMARCA,CUTERVO,SANTA CRUZ,60611,COMUNIDAD DE AMBATO,...,29/06/2006,IDELSO MENOR CHAVEZ,ACTIVADO,,EN FUNCIONAMIENTO,,-78.858380,-6.133523,1724.0,NaN
2,GOBIERNO REGIONAL,99,SANTA ISABEL DE YUMBATURO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LORETO,LORETO,PARINARI,160302,ACTUALIZAR,...,01/01/1900,JULIO TONY CAITIMARI MACHOA,ACTIVADO,,EN FUNCIONAMIENTO,,-74.258139,-4.581509,124.0,NaN
6,MINSA,7278,PUESTO DE SALUD HEROES DEL CENEPA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LIMA,LIMA,VILLA EL SALVADOR,150142,"JR. HEROES DEL CENEPA MZ C LT 20, ALT DE PANAM...",...,01/02/2008,BASTIDAS CAMARENA HUBERT ALEX . .,ACTIVADO,,EN FUNCIONAMIENTO,,-76.930608,-12.248699,NaN,NaN
12,GOBIERNO REGIONAL,5460,NUEVA BETANIA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,UCAYALI,CORONEL PORTILLO,CALLERIA,250101,OTROS CC.NN. NUEVA BETANIA S/N NÚMERO S/N DIST...,...,20/12/2000,MARIANA VASQUEZ PINEDO,ACTIVADO,,EN FUNCIONAMIENTO,,-74.296531,-8.398366,177.0,NaN
15,GOBIERNO REGIONAL,6431,PONGO ISLA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,SAN MARTIN,SAN MARTIN,HUIMBAYOC,220907,OTROS CP PONGO ISLA DISTRITO HUIMBAYOC PROVINC...,...,14/09/2001,MIRLANDA MOZOMBITE BARDALES,ACTIVADO,,EN FUNCIONAMIENTO,,-75.885812,-6.438298,172.1,NaN


### Número de hospitales operativos para Lima dentro de un área de influencia de 10 km

In [125]:
#1) Preparamos el centroide de Lima como GeoDataFrame (WGS84) 
gdf_cent_lima = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy([lon_lima], [lat_lima]), crs=4326
)

In [126]:
#2) Proyectar a UTM 18S (Lima) para medir en metros 
cent_utm = gdf_cent_lima.to_crs(32718)
hosp_utm = gdf_hosp.to_crs(32718)

In [127]:
# 3) Buffer de 10 km y conteo de hospitales dentro 
buf10 = cent_utm.buffer(10_000).iloc[0]                    # 10 km
dentro_utm = hosp_utm[hosp_utm.within(buf10)]
n_hosp_lima = int(len(dentro_utm))
print(f"Hospitales operativos ≤10 km del centroide de Lima: {n_hosp_lima}")

Hospitales operativos ≤10 km del centroide de Lima: 5


In [128]:
# Añadimos al mapa: círculo 10 km + hospitales dentro 
# círculo en metros (folium usa WGS84 para la ubicación, pero radio en metros)
fm.Circle([lat_lima, lon_lima], radius=10_000,
          color="#d73027", weight=2, fill=False,
          tooltip=f"Buffer 10 km – Lima ({n_hosp_lima} hosp)"
         ).add_to(m_lima)

In [129]:
# puntos de hospitales dentro del buffer
dentro_wgs84 = dentro_utm.to_crs(4326)
for _, r in dentro_wgs84.iterrows():
    fm.CircleMarker([r.geometry.y, r.geometry.x], radius=3,
                    color="#2166ac", fill=True, fill_opacity=0.9,
                    tooltip=r.get("NOMBRE DEL ESTABLECIMIENTO","Hospital")
                   ).add_to(m_lima)

m_lima

### Número de hospitales operativos para Loreto dentro de un área de influencia de 10 km .

In [130]:
#1) Preparamos el centroide de Loreto como GeoDataFrame (WGS84) 
gdf_cent_loreto = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy([lon_loreto], [lat_loreto]), crs=4326
)

In [131]:
#2) Proyectamos a UTM 18S (Lima) para medir en metros 
cent_utm = gdf_cent_loreto.to_crs(32718)
hosp_utm = gdf_hosp.to_crs(32718)

In [132]:
# 3) Buffer de 10 km y conteo de hospitales dentro 
buf10 = cent_utm.buffer(10_000).iloc[0]                    # 10 km
dentro_utm = hosp_utm[hosp_utm.within(buf10)]
n_hosp_loreto = int(len(dentro_utm))
print(f"Hospitales operativos ≤10 km del centroide de Loreto: {n_hosp_loreto}")

Hospitales operativos ≤10 km del centroide de Loreto: 0


In [133]:
# Añadimos al mapa: círculo 10 km + hospitales dentro 
# círculo en metros (folium usa WGS84 para la ubicación, pero radio en metros)
fm.Circle([lat_loreto, lon_loreto], radius=10_000,
          color="#d73027", weight=2, fill=False,
          tooltip=f"Buffer 10 km – Lima ({n_hosp_lima} hosp)"
         ).add_to(m_loreto)

In [134]:
# puntos de hospitales dentro del buffer
dentro_wgs84 = dentro_utm.to_crs(4326)
for _, r in dentro_wgs84.iterrows():
    fm.CircleMarker([r.geometry.y, r.geometry.x], radius=3,
                    color="#2166ac", fill=True, fill_opacity=0.9,
                    tooltip=r.get("NOMBRE DEL ESTABLECIMIENTO","Hospital")
                   ).add_to(m_loreto)

m_loreto

## Para la región LIMA, indentificamos el centro poblacional con menos hospitales cercanos (aislamiento).

In [155]:
# 1) Pasamos a UTM 18S (Lima) para medir en metros
cp_utm = ccpp_lima.to_crs(32718).copy()
hp_utm = gdf_hosp.to_crs(32718).copy()

In [156]:
# 2) Crear buffers de 10 km alrededor de cada centro poblado
cp_utm = cp_utm.reset_index(drop=False).rename(columns={"index": "cp_id"})
buf = cp_utm[["cp_id", "geometry"]].copy()
buf["geometry"] = buf.geometry.buffer(10_000)   # 10 km

In [157]:
# 3) Join espacial: hospitales dentro de cada buffer -> conteo por centro
join = gpd.sjoin(hp_utm, buf.set_geometry("geometry"), predicate="within", how="left")
conteo = join.groupby("cp_id").size().rename("n_hosp_10km")

In [158]:
# 4) Resultado por centro
res = cp_utm.merge(conteo, on="cp_id", how="left")
res["n_hosp_10km"] = res["n_hosp_10km"].fillna(0).astype(int)

In [159]:
# 5) Elegir columna de nombre del centro poblado
nombre_col = next((c for c in ["NOM_POBLAD","NOMBRE","N_BUSQDA","NOM_CCPP"] if c in res.columns), None)
if nombre_col is None:
    nombre_col = res.columns[0]  # fallback

In [160]:
# 6) Centro poblacional MÁS aislado (menor número de hospitales ≤10 km)
aislado = res.sort_values(["n_hosp_10km", nombre_col], ascending=[True, True]).iloc[0]

print(f"Centro poblacional más aislado en Lima (≤10 km): {aislado[nombre_col]}")
print(f"Hospitales operativos en 10 km: {aislado['n_hosp_10km']}")

Centro poblacional más aislado en Lima (≤10 km): ACACHI
Hospitales operativos en 10 km: 0


In [161]:
##Ploteamos 

In [166]:
ccpp_lima = ccpp_lima[ccpp_lima.NOM_POBLAD == "ACACHI"]
ccpp_lima

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
116170,120240,ACACHI,INEI,None,None,YAUYOS,YAUYOS,LIMA,None,None,-76.20807,-76.20807,None,POINT (-76.20807 -12.73005)


In [179]:
from geopy.distance import geodesic

In [180]:
# Coordenadas del centroide de ACACHI
lat_acachi, lon_acachi = -12.73005, -76.20807

In [190]:
# Creamod mapa centrado en ACACHI
m_acachi = fm.Map(location=[lat_acachi, lon_acachi], zoom_start=11, tiles="OpenStreetMap")

In [182]:
# Agregar marcador del centroide
fm.Marker(
    [lat_acachi, lon_acachi],
    popup="<b>Centro poblado: ACACHI</b>",
    tooltip="Centroide ACACHI",
    icon=fm.Icon(color="blue", icon="info-sign")
).add_to(m_acachi)

In [183]:
# Dibujar el círculo de 10 km
fm.Circle(
    [lat_acachi, lon_acachi],
    radius=10_000,   # en metros
    color="red",
    weight=2,
    fill=False,
    tooltip="Área de influencia (10 km)"
).add_to(m_acachi)

In [184]:
# Filtrar hospitales dentro de los 10 km
hosp_cercanos = []
for _, row in gdf_hosp.iterrows():
    hosp_point = (row.geometry.y, row.geometry.x)  # (lat, lon)
    centroide_point = (lat_acachi, lon_acachi)
    dist = geodesic(centroide_point, hosp_point).km
    
    if dist <= 10:
        hosp_cercanos.append(row)
        fm.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_color="green",
            popup=f"Hospital: {row['Nombre'] if 'Nombre' in row else 'SIN NOMBRE'}<br>Distancia: {dist:.2f} km",
            tooltip="Hospital operativo"
        ).add_to(m_acachi)

# Mostrar mapa
m_acachi

## Para la región LIMA, indentificamos el centro poblacional con mas hospitales cercanos (concentracion).

In [186]:
# Centro poblacional con MÁS hospitales (≤10 km)
mas_hosp = res.sort_values(["n_hosp_10km", nombre_col], ascending=[False, True]).iloc[0]

print(f"Centro poblacional con MÁS hospitales en Lima (≤10 km): {mas_hosp[nombre_col]}")
print(f"Hospitales operativos en 10 km: {mas_hosp['n_hosp_10km']}")

Centro poblacional con MÁS hospitales en Lima (≤10 km): BARRIO OBRERO INDUSTRIAL
Hospitales operativos en 10 km: 241


In [188]:
ccpp = ccpp[ccpp.NOM_POBLAD == "BARRIO OBRERO INDUSTRIAL"]
ccpp

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
115876,119882,BARRIO OBRERO INDUSTRIAL,INEI,None,None,SAN MARTIN DE PORRES,LIMA,LIMA,None,None,-77.04702,-77.04702,None,POINT (-77.04702 -12.03038)


In [195]:
# Coordenadas del centroide de BARRIO INDUSTRIAL
lat_industrial, lon_industrial = -12.03038, -77.04702

In [197]:
# Creamos mapa centrado en BARRIO OBRERO INDUSTRIAL
m_industrial = fm.Map(location=[lat_industrial, lon_industrial], 
                      zoom_start=11, 
                      tiles="OpenStreetMap")

In [198]:
# Agregar marcador del centroide
fm.Marker(
    [lat_industrial, lon_industrial],
    popup="<b>Centro poblado: BARRIO OBRERO INDUSTRIAL</b>",
    tooltip="Centroide BARRIO OBRERO INDUSTRIAL",
    icon=fm.Icon(color="blue", icon="info-sign")
).add_to(m_industrial)

In [199]:
# Dibujar el círculo de 10 km
fm.Circle(
    [lat_industrial, lon_industrial],
    radius=10_000,   # en metros
    color="red",
    weight=2,
    fill=False,
    tooltip="Área de influencia (10 km)"
).add_to(m_industrial)

In [200]:
# Filtrar hospitales dentro de los 10 km
hosp_cercanos = []
for _, row in gdf_hosp.iterrows():
    hosp_point = (row.geometry.y, row.geometry.x)  # (lat, lon)
    centroide_point = (lat_industrial, lon_industrial)
    dist = geodesic(centroide_point, hosp_point).km
    
    if dist <= 10:
        hosp_cercanos.append(row)
        fm.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_color="green",
            popup=f"Hospital: {row['Nombre'] if 'Nombre' in row else 'SIN NOMBRE'}<br>Distancia: {dist:.2f} km",
            tooltip="Hospital operativo"
        ).add_to(m_industrial)

# Mostramos mapa
m_industrial

## Para la región LORETO, indentificamos el centro poblacional con menos hospitales cercanos (aislamiento).

In [203]:
import geopandas as gpd

# 1) Proyectar a UTM 19S (Loreto) para medir en metros
cp_utm = ccpp_loreto.to_crs(32719).reset_index(drop=True)
hp_utm = gdf_hosp.to_crs(32719)

# 2) Buffer de 10 km por centro poblado
buf = gpd.GeoDataFrame({"cp_id": cp_utm.index, "geometry": cp_utm.geometry.buffer(10_000)}, crs=32719)

# 3) Join espacial: hospitales dentro de cada buffer -> conteo por centro
join = gpd.sjoin(hp_utm, buf.set_geometry("geometry"), predicate="within", how="left")
conteo = join.groupby("cp_id").size().rename("n_hosp_10km")

# 4) Resultado por centro (0 si no hay hospitales)
res = cp_utm.copy()
res["n_hosp_10km"] = conteo.reindex(res.index, fill_value=0).astype(int)

# 5) Columna de nombre del centro poblado
nombre_col = next((c for c in ["NOM_POBLAD","NOMBRE","N_BUSQDA","NOM_CCPP"] if c in ccpp_loreto.columns),
                  ccpp_loreto.columns[0])

# 6) Centro poblacional MÁS aislado en Loreto (menor número de hospitales ≤10 km)
aislado_loreto = res.sort_values(["n_hosp_10km", nombre_col], ascending=[True, True]).iloc[0]

print(f"Centro poblacional más aislado en Loreto (≤10 km): {aislado_loreto[nombre_col]}")
print(f"Hospitales operativos en 10 km: {aislado_loreto['n_hosp_10km']}")


Centro poblacional más aislado en Loreto (≤10 km): 1 DE FEBRERO
Hospitales operativos en 10 km: 0


In [206]:
ccpp_loreto = ccpp_loreto[ccpp_loreto.NOM_POBLAD == "1 DE FEBRERO"]
ccpp_loreto

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
120232,124706,1 DE FEBRERO,INEI,None,None,NAUTA,LORETO,LORETO,None,None,-73.56656,-73.56656,None,POINT (-73.56656 -4.36734)


In [207]:
# Coordenadas del centroide de 1 DE FEBRERO
lat_febrero, lon_febrero = -4.36734, -73.56656

# Creamos mapa centrado en 1 DE FEBRERO
m_febrero = fm.Map(location=[lat_febrero, lon_febrero], zoom_start=11, tiles="OpenStreetMap")

# Agregar marcador del centroide
fm.Marker(
    [lat_febrero, lon_febrero],
    popup="<b>Centro poblado: ACACHI</b>",
    tooltip="Centroide ACACHI",
    icon=fm.Icon(color="blue", icon="info-sign")
).add_to(m_febrero)
# Dibujar el círculo de 10 km
fm.Circle(
    [lat_febrero, lon_febrero],
    radius=10_000,   # en metros
    color="red",
    weight=2,
    fill=False,
    tooltip="Área de influencia (10 km)"
).add_to(m_febrero)
# Filtrar hospitales dentro de los 10 km
hosp_cercanos = []
for _, row in gdf_hosp.iterrows():
    hosp_point = (row.geometry.y, row.geometry.x)  # (lat, lon)
    centroide_point = (lat_febrero, lon_febrero)
    dist = geodesic(centroide_point, hosp_point).km
    
    if dist <= 10:
        hosp_cercanos.append(row)
        fm.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_color="green",
            popup=f"Hospital: {row['Nombre'] if 'Nombre' in row else 'SIN NOMBRE'}<br>Distancia: {dist:.2f} km",
            tooltip="Hospital operativo"
        ).add_to(m_acachi)

# Mostrar mapa
m_febrero

## Para la región LORETO, indentificamos el centro poblacional con mas hospitales cercanos (concentración).

In [208]:
# Centro poblacional con MÁS hospitales en Loreto (≤10 km)
mas_hosp_loreto = res.sort_values(["n_hosp_10km", nombre_col], ascending=[True, True]).iloc[-1]

print(f"Centro poblacional con MÁS hospitales en Loreto (≤10 km): {mas_hosp_loreto[nombre_col]}")
print(f"Hospitales operativos en 10 km: {mas_hosp_loreto['n_hosp_10km']}")


Centro poblacional con MÁS hospitales en Loreto (≤10 km): TRES DE OCTUBRE
Hospitales operativos en 10 km: 28


In [233]:
ccpp_loreto = ccpp_loreto[ccpp_loreto.NOM_POBLAD == "TRES DE OCTUBRE"]
ccpp_loreto

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
119053,123389,TRES DE OCTUBRE,INEI,1601130012,CASERÍO,SAN JUAN BAUTISTA,MAYNAS,LORETO,2263,Centro Poblado Menor,-73.28708,-73.28708,TRES DE OCTUBRE,POINT (-73.28708 -3.78232)


In [235]:
# Coordenadas del centroide de TRES DE OCTUBRE
lat_octubre, lon_octubre = -3.78232, -73.28708

In [237]:
# Creamos mapa centrado en TRES DE OCTUBRE
m_octubre = fm.Map(location=[lat_octubre, lon_octubre], 
                      zoom_start=11, 
                      tiles="OpenStreetMap")

In [238]:
# Agregar marcador del centroide
fm.Marker(
    [lat_octubre, lon_octubre],
    popup="<b>Centro poblado: BARRIO OBRERO INDUSTRIAL</b>",
    tooltip="Centroide BARRIO OBRERO INDUSTRIAL",
    icon=fm.Icon(color="blue", icon="info-sign")
).add_to(m_octubre)

In [239]:
# Dibujar el círculo de 10 km
fm.Circle(
    [lat_octubre, lon_octubre],
    radius=10_000,   # en metros
    color="red",
    weight=2,
    fill=False,
    tooltip="Área de influencia (10 km)"
).add_to(m_octubre)

In [240]:
# Filtrar hospitales dentro de los 10 km
hosp_cercanos = []
for _, row in gdf_hosp.iterrows():
    hosp_point = (row.geometry.y, row.geometry.x)  # (lat, lon)
    centroide_point = (lat_octubre, lon_octubre)
    dist = geodesic(centroide_point, hosp_point).km
    
    if dist <= 10:
        hosp_cercanos.append(row)
        fm.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_color="green",
            popup=f"Hospital: {row['Nombre'] if 'Nombre' in row else 'SIN NOMBRE'}<br>Distancia: {dist:.2f} km",
            tooltip="Hospital operativo"
        ).add_to(m_octubre)

# Mostramos mapa
m_octubre